### IMPORTS

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, precision_score, recall_score, f1_score

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Dense, LSTM, RepeatVector, TimeDistributed, 
                                     MultiHeadAttention, LayerNormalization, Add, 
                                     Conv1D, GlobalAveragePooling1D)

from tensorflow.keras.callbacks import EarlyStopping
from einops import rearrange

import os
import gc
import tensorflow.keras.backend as K

### DEVICE SETUP

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        tf.config.set_visible_devices(gpus[0], 'GPU')
        print("✅ GPU is available and will be used.")
    except RuntimeError as e:
        print(e)
else:
    print("⚠️ No GPU detected, running on CPU.")

✅ GPU is available and will be used.


### LOAD AND PREPROCESS DATA

In [3]:
file_path = '../forecast_datasets/training_set_labeled.csv'
df = pd.read_csv(file_path, delimiter=',')
df['DateTime'] = pd.to_datetime(df['DateTime'], errors='coerce')
df.set_index('DateTime', inplace=True)
df.drop(columns=['labels'], inplace=True)

# Normalize training set
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df.values)
df_scaled = pd.DataFrame(scaled_data, index=df.index, columns=df.columns).astype(np.float32)

print(f"✅ Scaled dataset shape: {df_scaled.shape}")


file_path2 = '../forecast_datasets/test_set_labeled.csv'
df_test = pd.read_csv(file_path2, delimiter=',')
df_test['DateTime'] = pd.to_datetime(df_test['DateTime'], errors='coerce')
df_test.set_index('DateTime', inplace=True)
df_test.drop(columns=['labels'], inplace=True)

# Normalize test set
scaled_test_data = scaler.transform(df_test.values)
df_test_scaled = pd.DataFrame(scaled_test_data, index=df_test.index, columns=df_test.columns).astype(np.float32)

print(f"✅ Scaled dataset shape: {df_test_scaled.shape}")

✅ Scaled dataset shape: (62174, 26)
✅ Scaled dataset shape: (40436, 26)


### CONFIGURATION

In [4]:
INPUT_STEPS = 10
FORECAST_STEPS = 10
EMBED_DIMS = [128]
NB_HEADS = [4]
PATCH_LEN_LIST = [5]
NB_LAYERS = [1]

# Tuning parameters
EPOCHS_LIST = [20]
BATCH_SIZES = [1024]
TEST_SIZE_POURCENTAGE = 0.4
WINDOW_SIZE_SIMULATION = 10  # 30 mins window

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [5]:
# Split for test set
split_idx_test = int(TEST_SIZE_POURCENTAGE * len(df_test_scaled))
test_set_intermediaire = df_test_scaled.iloc[:split_idx_test]

test_data = test_set_intermediaire.tail(15600).reset_index(drop=True)

print(f"✅ Forecast model-Testing samples: {test_data.shape}")

# ========================
# 4. SEQUENTIAL TRAIN/TEST SPLIT
# ========================
train_data = df_scaled
print(f"✅ Training samples: {len(train_data)}, Testing samples: {len(test_data)}")

# ========================
# 5. CREATE SEQUENCES
# ========================
def create_sequences(data, input_steps, forecast_steps):
    X, y = [], []
    for i in range(len(data) - input_steps - forecast_steps):
        X.append(data[i:i+input_steps])
        y.append(data[i+input_steps:i+input_steps+forecast_steps])
    return np.array(X, dtype=np.float32), np.array(y, dtype=np.float32)

X_train_seq, y_train_seq = create_sequences(train_data.values, INPUT_STEPS, FORECAST_STEPS)
X_test_seq, y_test_seq = create_sequences(test_data.values, INPUT_STEPS, FORECAST_STEPS)

print(f"✅ Training sequences: {X_train_seq.shape}, Testing sequences: {X_test_seq.shape}")

✅ Forecast model-Testing samples: (15600, 26)
✅ Training samples: 62174, Testing samples: 15600
✅ Training sequences: (62154, 10, 26), Testing sequences: (15580, 10, 26)


In [6]:
# ========================
# 6. BUILD PATCHTST MODEL (CORRECTED)
# ========================
def build_patchtst(input_steps, num_features, patch_len, embed_dim, num_heads, num_layers):
    assert input_steps % patch_len == 0, f"Input steps ({input_steps}) must be divisible by patch length ({patch_len})"
    num_patches = input_steps // patch_len

    inp = Input(shape=(input_steps, num_features))
    
    # Channel Independence
    x = tf.reshape(inp, (-1, input_steps, 1))  # (batch*num_features, timesteps, 1)
    
    # Patching
    x = tf.reshape(x, (-1, num_patches, patch_len))  # (batch*num_features, num_patches, patch_len)
    x = Dense(embed_dim)(x)  # (batch*num_features, num_patches, embed_dim)

    # Transformer Blocks
    for _ in range(num_layers):
        # Self-Attention
        attn = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(x, x)
        x = LayerNormalization()(Add()([x, attn]))
        
        # Feed Forward Network
        ffn = Dense(embed_dim * 4, activation='gelu')(x)
        ffn = Dense(embed_dim)(ffn)
        x = LayerNormalization()(Add()([x, ffn]))

    # Output Head
    x = x[:, -1, :]  # Last patch only (batch*num_features, embed_dim)
    x = tf.reshape(x, (-1, num_features, embed_dim))  # (batch, num_features, embed_dim)
    
    # Ensure output matches FORECAST_STEPS
    x = Dense(FORECAST_STEPS)(x)  # (batch, num_features, forecast_steps)
    out = tf.transpose(x, [0, 2, 1])  # (batch, forecast_steps, num_features)

    model = Model(inp, out)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mse')
    return model

In [7]:
# # ========================
# # 7. TRAINING + TUNING (CORRECTED)
# # ========================
# # First verify shapes
# print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
# print(f"Input steps: {INPUT_STEPS}, Forecast steps: {FORECAST_STEPS}, Features: {X_train.shape[2]}")

# # Test model output shape
# test_model = build_patchtst(INPUT_STEPS, X_train.shape[2])
# test_pred = test_model.predict(X_train[:1])
# print(f"Test prediction shape: {test_pred.shape} (should match {y_train[:1].shape})")

# assert test_pred.shape == y_train[:1].shape, \
#     f"Shape mismatch! Model outputs {test_pred.shape} but y_train has {y_train[:1].shape}"

# best_val_mse = np.inf
# best_model = None
# history_records = []

# for epochs in EPOCHS_LIST:
#     for batch_size in BATCH_SIZES:
#         print(f"\n🔵 Training with epochs={epochs}, batch_size={batch_size}")
        
#         model = build_patchtst(INPUT_STEPS, X_train.shape[2])
#         es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        
#         history = model.fit(
#             X_train, y_train,
#             validation_split=0.1,
#             epochs=epochs,
#             batch_size=batch_size,
#             callbacks=[es],
#             verbose=1,
#             shuffle=False
#         )
        
#         val_preds = model.predict(X_test, batch_size=batch_size)
        
#         # Flatten all except batch dimension for metrics
#         val_mse = mean_squared_error(
#             y_test.reshape(-1, FORECAST_STEPS * X_test.shape[2]), 
#             val_preds.reshape(-1, FORECAST_STEPS * X_test.shape[2])
#         )
#         val_mae = mean_absolute_error(
#             y_test.reshape(-1, FORECAST_STEPS * X_test.shape[2]),
#             val_preds.reshape(-1, FORECAST_STEPS * X_test.shape[2])
#         )

#         print(f"✅ Val MSE: {val_mse:.5f}, MAE: {val_mae:.5f}")

#         history_records.append({
#             "epochs": epochs,
#             "batch_size": batch_size,
#             "val_mse": val_mse,
#             "val_mae": val_mae,
#             "best_epoch": np.argmin(history.history['val_loss']) + 1
#         })

#         if val_mse < best_val_mse:
#             best_val_mse = val_mse
#             best_model = model
#             print("🏆 New best model!")

# # Save results
# history_df = pd.DataFrame(history_records)
# history_df.to_csv("patchtst_tuning_history.csv", index=False)
# print("\n📋 Tuning Results:")
# print(history_df.sort_values('val_mse'))

# if best_model:
#     best_model.save("best_patchtst_model.h5")
#     print("\n✅ Best model saved with:")
#     print(f"- Val MSE: {best_val_mse:.5f}")
#     print(f"- Best config: {history_df.loc[history_df['val_mse'].idxmin()]}")

best_val_mse = np.inf
best_model = None
history_records = []

for epochs in EPOCHS_LIST:
    for batch_size in BATCH_SIZES:
        for patch_len in PATCH_LEN_LIST:
            for embed_dim in EMBED_DIMS:
                for nb_head in NB_HEADS:
                    for nb_layer in NB_LAYERS:
                        print(f"\n🔵 Training model with epochs={epochs}, batch_size={batch_size}")
                        
                        model = build_patchtst(INPUT_STEPS, X_train_seq.shape[2], patch_len, embed_dim, nb_head, nb_layer)
                        es = EarlyStopping(patience=5, restore_best_weights=True)

                        history = model.fit(X_train_seq, y_train_seq,
                                            validation_split=0.1,
                                            epochs=epochs,
                                            batch_size=batch_size,
                                            callbacks=[es],
                                            verbose=1,
                                            shuffle=False)
                        
                        val_preds = model.predict(X_train_seq, batch_size=batch_size)
                        val_mse = mean_squared_error(y_train_seq.reshape(-1), val_preds.reshape(-1))
                        val_mae = mean_absolute_error(y_train_seq.reshape(-1), val_preds.reshape(-1))

                        print(f"✅ Validation MSE: {val_mse:.5f}, MAE: {val_mae:.5f}")

                        history_records.append({
                            "epochs": epochs,
                            "batch_size": batch_size,
                            "EMBED_DIMS": embed_dim, 
                            "NB_HEADS": nb_head,
                            "PATCH_LEN": patch_len,
                            "NB_LAYERS" : nb_layer,
                            "val_mse": val_mse,
                            "val_mae": val_mae
                        })

                        if val_mse < best_val_mse:
                            best_val_mse = val_mse
                            best_model = model

# Save history
history_df = pd.DataFrame(history_records)
history_df.to_csv("standard_patchtst_tuning_history.csv", index=False)
print("\n📋 Tuning Results Summary:")
print(history_df)

# Save best model
best_model.save("best_standard_patchtst_forecaster.h5")
print("\n✅ Best PatchTST model saved.")



🔵 Training model with epochs=20, batch_size=1024
Epoch 1/20
55/55 [==============================] - 15s 185ms/step - loss: 0.1476 - val_loss: 0.0548
Epoch 2/20
55/55 [==============================] - 9s 172ms/step - loss: 0.0531 - val_loss: 0.0559
Epoch 3/20
55/55 [==============================] - 10s 173ms/step - loss: 0.0510 - val_loss: 0.0554
Epoch 4/20
55/55 [==============================] - 10s 177ms/step - loss: 0.0496 - val_loss: 0.0557
Epoch 5/20
55/55 [==============================] - 10s 184ms/step - loss: 0.0484 - val_loss: 0.0576
Epoch 6/20
61/61 [==============================] - 4s 57ms/step
✅ Validation MSE: 0.06242, MAE: 0.20500

📋 Tuning Results Summary:
   epochs  batch_size  EMBED_DIMS  NB_HEADS  PATCH_LEN  NB_LAYERS   val_mse  \
0      20        1024         128         4          5          1  0.062422   

    val_mae  
0  0.204999  

✅ Best PatchTST model saved.


In [8]:
K.clear_session()
gc.collect()

2144

### REAL-TIME SIMULATION ON TEST SET

In [9]:
simulation_X, simulation_y = create_sequences(test_data.values, INPUT_STEPS, FORECAST_STEPS)

forecast_list = []
true_windows = []

for i in range(0, len(simulation_X), WINDOW_SIZE_SIMULATION):
    window_X = simulation_X[i:i+1]
    window_y_true = simulation_y[i]

    y_pred_future = best_model.predict(window_X,batch_size=1024, verbose=1)[0]

    forecast_list.append(y_pred_future)
    true_windows.append(window_y_true)

print("\n✅ Real-time simulation complete.")

1/1 [==============================] - 0s 22ms/step

✅ Real-time simulation complete.


### EVALUATION

In [11]:
# Forecasting metrics
y_pred_all = np.vstack(forecast_list)
y_true_all = np.vstack(true_windows)

forecast_mse = mean_squared_error(y_true_all.reshape(-1), y_pred_all.reshape(-1))
forecast_mae = mean_absolute_error(y_true_all.reshape(-1), y_pred_all.reshape(-1))

print(f"\n📈 Forecasting Evaluation on Test:")
print(f"MSE: {forecast_mse:.5f}")
print(f"MAE:  {forecast_mae:.5f}")

# ========================
# SAVE METRICS
# ========================

metrics_results = {
    "Model": "Adapted PatchTST",
    "Forecast_MSE": forecast_mse,
    "Forecast_MAE": forecast_mae
}


# Save tuning history
metrics_df = pd.DataFrame([metrics_results])
metrics_df.to_csv("adapted_patchTST_test_evaluation.csv", index=False)
print("\n📋 Test Results Summary:")
print(metrics_df)


📈 Forecasting Evaluation on Test:
MSE: 0.16723
MAE:  0.31895

📋 Test Results Summary:
              Model  Forecast_MSE  Forecast_MAE
0  Adapted PatchTST      0.167232      0.318949
